In [ ]:
import torch
import json
import os

layer = 5  # Manually set layer value

# Function to process a single folder
def process_folder(base_path, folder_name):
    full_path = os.path.join(base_path, folder_name)
    model_name = os.path.join(full_path, "ae.pt")

    if not os.path.exists(model_name):
        print(f"Model file missing: {model_name}")
        return

    model = torch.load(model_name)

    hidden_dim = model['encoder.weight'].shape[0]
    input_dim = model['encoder.weight'].shape[1]

    assert hidden_dim % input_dim == 0

    expansion_factor = int(hidden_dim / input_dim)
    config_path = os.path.join(full_path, "config.json")

    if not os.path.exists(config_path):
        print(f"Config file missing: {config_path}")
        return

    with open(config_path) as f:
        config = json.load(f)

    lr = format(config['trainer']['lr'], '.0e')
    l1 = format(config['trainer']['l1_penalty'], '.0e')

    if 'activation_dim' not in config['trainer']:
        config['trainer']['activation_dim'] = input_dim

    if 'dictionary_size' not in config['trainer']:
        config['trainer']['dictionary_size'] = hidden_dim

    config['trainer']['layer'] = layer

    with open(config_path, 'w') as f:
        json.dump(config, f, indent=4)

    new_folder_name = os.path.join(base_path, f"ef={expansion_factor}_lr={lr}_l1={l1}_layer={layer}")

    print(f"Renaming {full_path} to {new_folder_name}")
    os.rename(full_path, new_folder_name)

# Process current directory and one level deep subdirectories
for folder in os.listdir():
    if os.path.isdir(folder):
        process_folder('.', folder)  # Process folders in the current directory
        for subfolder in os.listdir(folder):  # Process subfolders
            subfolder_path = os.path.join(folder, subfolder)
            if os.path.isdir(subfolder_path):
                process_folder(folder, subfolder)
